In [1]:
!pip install "chronos[training] @ git+https://github.com/amazon-science/chronos-forecasting.git"

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-install-w4qc1vkn/chronos_1e942efc993d4ae98c0f77d7acd9fec7
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-install-w4qc1vkn/chronos_1e942efc993d4ae98c0f77d7acd9fec7
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit 050d600f6430c0608904ae6e3a90c1bf6b93cf84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)

In [2]:
!git clone https://github.com/amazon-science/chronos-forecasting.git

Cloning into 'chronos-forecasting'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 278 (delta 102), reused 70 (delta 60), pack-reused 132
Receiving objects: 100% (278/278), 842.84 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [4]:
!cd chronos-forecasting/

In [36]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Union
from gluonts.dataset.arrow import ArrowWriter
import torch

In [35]:
def convert_csv_to_arrow(
    csv_url: str,
    arrow_path: Union[str, Path],
    store_id: int
):
    # Read CSV file from URL
    df = pd.read_csv(csv_url)

    # Filter data for the specified store
    store_data = df[df['Store'] == store_id]

    # Convert 'Date' column to datetime
    store_data['Date'] = pd.to_datetime(store_data['Date'])

    # Get the start time (assuming all data for a store starts at the same time)
    start_time = store_data['Date'].min()

    # Extract Weekly_Sales column and convert to list of numpy arrays
    time_series = [store_data['Weekly_Sales'].values]

    # Prepare dataset
    dataset = [
        {"start": start_time, "target": ts} for ts in time_series
    ]

    # Write dataset to Arrow format
    ArrowWriter(compression="lz4").write_to_file(dataset, path=arrow_path)

csv_url = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/main/data/walmart_sales_small.csv"
convert_csv_to_arrow(csv_url, "sales_data_store1.arrow", store_id=1)


<ipython-input-35-cd70f567d8c6>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  store_data['Date'] = pd.to_datetime(store_data['Date'])


In [15]:
!ls

chronos-forecasting   output		       sample_data
finetune_config.yaml  sales_data_store1.arrow  walmart_sales.arrow


In [16]:
import yaml

# Define the configuration as a dictionary
config = {
    'training_data_paths': [
        "/content/sales_data_store1.arrow"
    ],
    'probability': [0.9],
    'context_length': 64,
    'prediction_length': 8,
    'min_past': 60,
    'max_steps': 500,
    'save_steps': 250,
    'log_steps': 100,
    'per_device_train_batch_size': 32,
    'learning_rate': 0.001,
    'optim': 'adamw_torch_fused',
    'num_samples': 20,
    'shuffle_buffer_length': 100,
    'gradient_accumulation_steps': 1,
    'model_id': 'google/t5-efficient-small',
    'model_type': 'seq2seq',
    'random_init': False,
    'tie_embeddings': True,
    'output_dir': './output/',
    'tf32': True,
    'torch_compile': True,
    'tokenizer_class': 'MeanScaleUniformBins',
    'tokenizer_kwargs': {
        'low_limit': -15.0,
        'high_limit': 15.0
    },
    'n_tokens': 4096,
    'lr_scheduler_type': 'linear',
    'warmup_ratio': 0.0,
    'dataloader_num_workers': 1,
    'max_missing_prop': 0.9,
    'use_eos_token': True
}

# Specify the path to the YAML file
yaml_file_path = 'finetune_config.yaml'

# Write the dictionary to a YAML file
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(config, yaml_file, default_flow_style=False)

In [17]:
import os

# List all files in the current working directory
files_in_directory = os.listdir()

# Print the absolute path for each file
for file_name in files_in_directory:
    absolute_path = os.path.abspath(file_name)
    print(f"File: {file_name} | Absolute Path: {absolute_path}")


File: .config | Absolute Path: /content/.config
File: chronos-forecasting | Absolute Path: /content/chronos-forecasting
File: finetune_config.yaml | Absolute Path: /content/finetune_config.yaml
File: output | Absolute Path: /content/output
File: walmart_sales.arrow | Absolute Path: /content/walmart_sales.arrow
File: sales_data_store1.arrow | Absolute Path: /content/sales_data_store1.arrow
File: sample_data | Absolute Path: /content/sample_data


In [18]:
!CUDA_VISIBLE_DEVICES=0 python chronos-forecasting/scripts/training/train.py --config /content/finetune_config.yaml

2024-07-23 14:37:19.107818: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 14:37:19.107867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 14:37:19.109317: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 14:37:19.117006: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 14:37:20.205845: W tensorflow/comp

In [20]:
# List all files in the current working directory
files_in_directory = os.listdir()

# Print the absolute path for each file
for file_name in files_in_directory:
    absolute_path = os.path.abspath(file_name)
    print(f"File: {file_name} | Absolute Path: {absolute_path}")


File: .config | Absolute Path: /content/.config
File: chronos-forecasting | Absolute Path: /content/chronos-forecasting
File: finetune_config.yaml | Absolute Path: /content/finetune_config.yaml
File: output | Absolute Path: /content/output
File: walmart_sales.arrow | Absolute Path: /content/walmart_sales.arrow
File: sales_data_store1.arrow | Absolute Path: /content/sales_data_store1.arrow
File: sample_data | Absolute Path: /content/sample_data


In [29]:
!ls /content/output/run-1/checkpoint-final/

config.json  generation_config.json  model.safetensors	training_info.json


## Use finetuned model

In [34]:
url = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/main/data/walmart_sales_small.csv"

df = pd.read_csv(url, parse_dates=['Date'])
df = df.query('Store == 1')

In [37]:
def cross_validation_chronos(df, h, n_windows=4):

  lows = []
  medians = []
  highs = []

  for i in range(4, 0, -1):
    context = torch.tensor(df['Weekly_Sales'][:-(h*i)])

    predictions = pipeline.predict(
        context=context,
        prediction_length=h,
        num_samples=20
    )

    low, median, high = np.quantile(predictions[0].numpy(), [0.1, 0.5, 0.9], axis=0)

    lows.extend(low)
    medians.extend(median)
    highs.extend(high)

  return lows, medians, highs

In [39]:
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained("/content/output/run-1/checkpoint-final/", device_map='cuda', torch_dtype=torch.bfloat16)

In [41]:
lows, medians, highs = cross_validation_chronos(df, 8)

In [42]:
test_df = df[['Store', 'Date', 'Weekly_Sales']][-32:]

test_df['low'] = lows
test_df['chronos-finetuned'] = medians
test_df['high'] = highs

test_df.head()

,Store,Date,Weekly_Sales,low,chronos-finetuned,high
111,1,2012-03-23,1511068.07,1.505289e+06,1.658563e+06,1.898768e+06
112,1,2012-03-30,1649604.63,-7.778086e+05,1.578494e+06,1.898768e+06
113,1,2012-04-06,1899676.88,1.395480e+06,1.624247e+06,1.826706e+06
114,1,2012-04-13,1621031.70,1.387473e+06,1.567056e+06,1.898768e+06
115,1,2012-04-20,1521577.87,1.464110e+06,1.624247e+06,1.808405e+06


In [43]:
!pip install utilsforecast

In [44]:
from utilsforecast.losses import mae, smape
from utilsforecast.evaluation import evaluate

evaluation = evaluate(
    test_df,
    metrics=[mae, smape],
    models=['chronos-finetuned'],
    target_col='Weekly_Sales',
    id_col='Store'
)

evaluation

,Store,metric,chronos-finetuned
0,1,mae,63810.728031
1,1,smape,0.019902
